In [ ]:
# Linking to drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!pip install unbabel-comet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 2.2 MB/s 
     |████████████████████████████████| 9.5 MB 10.8 MB/s 
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
     |████████████████████████████████| 1.2 MB 53.4 MB/s 
     |████████████████████████████████| 4.4 MB 58.1 MB/s 
     |████████████████████████████████| 101 kB 14.5 MB/s 
     |████████████████████████████████| 409 kB 46.6 MB/s 
     |████████████████████████████████| 92 kB 13.5 MB/s 
     |████████████████████████████████| 585 kB 66.1 MB/s 
     |████████████████████████████████| 140 kB 75.0 MB/s 
     |████████████████████████████████| 596 kB 68.5 MB/s 
     |████████████████████████████████| 1.1 MB 59.7 MB/s 
     |████████████████████████████████| 6.6 MB 52.4 MB/s 
     |████████████████████████████████| 86 kB 7.0 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████

In [ ]:
from comet import download_model, load_from_checkpoint

In [ ]:
# Loading the drive
import os
os.chdir("/content/gdrive/Shareddrives/Low_Budget_MT")

In [ ]:
# Importing needed libraries for preprocessing and visualization
from my_helpers import split_srctgt, generating_BPE, run_model

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#! pip install torch==1.8.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install . --use-feature=in-tree-build

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/gdrive/Shareddrives/Low_Budget_MT/joeynmt
     |████████████████████████████████| 116 kB 26.9 MB/s 
     |████████████████████████████████| 507 kB 57.2 MB/s 
     |████████████████████████████████| 264 kB 68.4 MB/s 
     |████████████████████████████████| 103 kB 72.4 MB/s 
     |████████████████████████████████| 843 kB 65.7 MB/s 
     |████████████████████████████████| 57 kB 6.3 MB/s 
  Created wheel for joeynmt: filename=joeynmt-1.5.1-py3-none-any.whl size=86509 sha256=ce413a1d1870e7a3b9ad28be97e98ee058c7260a9c40f2c6210994b5f8e7086d
  Stored in directory: /tmp/pip-ephem-wheel-cache-8o25j_x3/wheels/ec/b0/4e/e15e388dd23d01f1c2b4065e1bb36099e5e22abe4f7147f706
  Created wheel for wrapt: filename=wrapt-1.11.1-cp37-cp37m-linux_x86_64.whl size=68423 sha256=d121f0c4c022628fb16f8e561711d31879fe38df40bf6921e51c9130e37d4a7a
  Stored in directory: /root/.cache/pip/wheels/4e/58/9d

In [ ]:
!pip install -U torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 12.5 MB/s 
     |████████████████████████████████| 831.4 MB 2.8 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.


In [ ]:
# Setting language and data file:
import pandas as pd
# 3. COMET
def comet(files, train_file, number):

  data = {}
  l  = []
  for i in files: 
    with open(i, "r") as src_file:
        x = src_file.read().splitlines()
        l.append(len(x))
        data[i] = x
  assert l.count(l[0]) == len(l), 'Check your files'

  df = pd.DataFrame(data)
  df.columns = ['source_sentence','target_sentence','scores']
  df["scores"] = pd.to_numeric(df["scores"])
  df = df.sort_values(by = ['scores'])
  print(df.head(1))

  return df.head(number), df.tail(df.shape[0]-number)

def comet_data_load(datafile, target_language, query, k, iteration):

  path = os.getcwd() 
  data_path = 'data/'+target_language
  train_file = '/content/gdrive/Shareddrives/Low_Budget_MT/data/sw/'+k+'/'+'swtrain.csv'
  fpath = os.path.join(path, data_path,k,query)
  print(path)
  print(data_path)
  print(fpath)

  if iteration== str(2):
    it = str(int(iteration)-1)
    train = pd.read_csv('/content/gdrive/Shareddrives/Low_Budget_MT/data/'+target_language+'/'+k+'/'+target_language+'train.csv')
    print(train.shape)
    t, rem = comet([fpath+'/1/rem.en', fpath+'/1/rem.'+target_language, fpath+'/1/scores'], train_file, 20000)
    print(t.shape, rem.shape)
    train_full = pd.concat([train,t])

    os.chdir(fpath)

    t.to_csv(iteration+'.csv', index=False)
    train_full.to_csv(query+'_train.csv', index=False)
    rem.to_csv(query+'_rem.csv',index=False)

    os.chdir(path)

  else:
    os.chdir(fpath)

    it = str(int(iteration)-1)

    train, rem = comet([fpath+'/'+it+'/rem.en', fpath+'/'+it+'/rem.'+target_language, fpath+'/'+it+'/scores'], train_file, 20000)
    print(train.shape)
    train.to_csv(iteration+'.csv', index=False)
    train.to_csv(query+'_train.csv',index=False, mode='a', header=False)
    rem.to_csv(query+'_rem.csv',index=False)

    os.chdir(path)

In [ ]:
def yhat(rem_file, src, tgt,k,iteration,M):
  path = '/content/gdrive/Shareddrives/Low_Budget_MT/data/'+tgt+'/'+k+'/comet/'+iteration
  df = pd.read_csv(rem_file)
  print(df.shape)

  os.chdir(path)
  print(os.getcwd())

  # Splitting x and y:
  os.environ['src'] = src
  os.environ['tgt']  = tgt
  with open("rem."+src, "w") as src_file, open("rem."+tgt, "w") as trg_file:
      for index, row in df.iterrows():
        src_file.write(str(row["source_sentence"])+"\n")
        trg_file.write(str(row["target_sentence"])+"\n")

  # Getting bpe for rem
  os.system(' subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < rem.$tgt > rem.bpe.$tgt')
  os.chdir('/content/gdrive/Shareddrives/Low_Budget_MT')
  print(os.getcwd())

  #translate rem
  os.environ["model_path"] = '/content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/'+k+'/'+M+'/config.yaml'
  os.environ["x_path"] = path+'/rem.bpe.'+tgt
  os.environ["yhat_path"] = path+'/yhat.'+src

  os.system('cd joeynmt; python -m joeynmt translate $model_path < $x_path > $yhat_path')


def comet_scoring(rem_file, src, tgt,k,iteration,M):
  path = '/content/gdrive/Shareddrives/Low_Budget_MT/data/'+tgt+'/'+k+'/comet/'+iteration
  print('Starting here')
  with open(path+'/rem.'+src, "r") as x_file, open(path+'/rem.bpe.'+tgt, "r") as y_file, open(path+'/yhat.'+src, "r") as z_file:
    x = x_file.read().splitlines()
    y = y_file.read().splitlines()
    z = z_file.read().splitlines()
    print(len(x),len(y),len(z))

  with open(path+'/rem.'+tgt, "r") as a_file:
    a = a_file.read().splitlines()

    print(len(a))

  f = [i for i,j in enumerate(z) if len(j)==0]

  if len(f) !=0:
    x = [i for j, i in enumerate(x) if j not in f]
    y = [i for j, i in enumerate(y) if j not in f]
    z = [i for j, i in enumerate(z) if j not in f]
    a = [i for j, i in enumerate(a) if j not in f]
    
    print(len(x),len(y),len(z))
  
  with open(path+'/rem.'+src, "w") as x_file, open(path+'/rem.bpe.'+tgt, "w") as y_file, open(path+'/yhat.'+src, "w") as z_file:
    for i in zip(x,y,z):
        x_file.write(i[0]+"\n")
        y_file.write(i[1]+"\n")
        z_file.write(i[2]+"\n")

  with open(path+'/rem.'+tgt, "w") as a_file:
    for i in a:
        a_file.write(i+"\n")

  assert len(x)==len(a)==len(z), "Error in translating"

  data = [{'src':i[0], 'mt':i[1]} for i in zip(x,y)]

  model_path = download_model("wmt21-comet-qe-mqm")
  model = load_from_checkpoint(model_path)

  seg_scores, sys_score = model.predict(data, batch_size=8, gpus=1)

  print(len(seg_scores))

  with open(path+'/scores', "w") as x_file:
    for i in seg_scores:
        x_file.write(str(i)+"\n")

  if os.path.exists(path+'/scores'):
    print('done')
  else:
    print('error')

# Kinyarwanda COMET K1


## Iteration 5

### AL Strategy and preprocessing

In [ ]:
comet_data_load('rwtrain.csv', 'rw', 'comet','k1', '2')

/content/gdrive/Shareddrives/Low_Budget_MT
data/rw
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k1/comet
(30000, 2)
                                         source_sentence  \
59487  Everything about the castle ​ — its multicolor...   

                                         target_sentence    scores  
59487  Ibyo bishushanyo byari iby’ibimasa bifite amab... -0.024238  
(20000, 3) (80000, 3)


In [ ]:
split_srctgt('/data/rw/k1/comet/comet_train.csv','en', 'rw', 'comet','k1', '2')

(50000, 3)


In [ ]:
generating_BPE('en', 'rw', 'comet', 'k1','2')

In [ ]:
# Swahili to English
run_model('rw', 'rw','en','transformer_reverse_enrw2_comet','comet','k1','2')
! cd joeynmt; python3 -m joeynmt train configs/k1/transformer_reverse_enrw2_comet.yaml

2022-09-03 21:45:00,151 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-09-03 21:45:01,185 - INFO - joeynmt.data - Loading training data...
2022-09-03 21:45:02,096 - INFO - joeynmt.data - Building vocabulary...
2022-09-03 21:45:02,387 - INFO - joeynmt.data - Loading dev data...
2022-09-03 21:45:02,403 - INFO - joeynmt.data - Loading test data...
2022-09-03 21:45:02,798 - INFO - joeynmt.data - Data loaded.
2022-09-03 21:45:02,798 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-09-03 21:45:03,003 - INFO - joeynmt.model - Enc-dec model built.
2022-09-03 21:45:07,342 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-09-03 21:45:07,711 - INFO - joeynmt.training - Total params: 12147712
2022-09-03 21:45:11,238 - INFO - joeynmt.helpers -                           cfg.name : transformer_reverse_enrw2_comet
2022-09-03 21:45:11,239 - INFO - joeynmt.helpers -                       cfg.data.src : rw
2022-09-03 21:45:11,239 - INFO - joeynmt.helpers -   

In [ ]:
# Output our validation accuracy
! cat "joeynmt/models/k1/transformer_reverse_enrw2_comet/validations.txt"

Steps: 600	Loss: 126285.07812	PPL: 95.38068	bleu: 0.12197	LR: 0.00100000	*
Steps: 1200	Loss: 113637.28125	PPL: 60.42435	bleu: 1.40586	LR: 0.00100000	*
Steps: 1800	Loss: 104153.67969	PPL: 42.91025	bleu: 1.51993	LR: 0.00100000	*
Steps: 2400	Loss: 97033.69531	PPL: 33.18628	bleu: 2.89620	LR: 0.00100000	*
Steps: 3000	Loss: 92425.46094	PPL: 28.10131	bleu: 3.30985	LR: 0.00100000	*
Steps: 3600	Loss: 89131.41406	PPL: 24.95134	bleu: 4.05180	LR: 0.00100000	*
Steps: 4200	Loss: 86327.32031	PPL: 22.54972	bleu: 4.77805	LR: 0.00100000	*
Steps: 4800	Loss: 84102.10938	PPL: 20.80951	bleu: 4.82009	LR: 0.00100000	*
Steps: 5400	Loss: 82270.58594	PPL: 19.47842	bleu: 5.49411	LR: 0.00100000	*
Steps: 6000	Loss: 80502.57812	PPL: 18.27430	bleu: 6.10204	LR: 0.00100000	*
Steps: 6600	Loss: 78975.75000	PPL: 17.29452	bleu: 6.31030	LR: 0.00100000	*
Steps: 7200	Loss: 78007.94531	PPL: 16.70085	bleu: 6.00559	LR: 0.00100000	*
Steps: 7800	Loss: 76569.01562	PPL: 15.85565	bleu: 7.79296	LR: 0.00100000	*
Steps: 8400	Loss: 75783

In [ ]:
# Reloading configuration file
ckpt_number = 61200
config = run_model('rw', 'rw','en','transformer_reverse_enrw2_comet','comet','k1','2')
reload_config = config.replace(
    f'#load_model: path ', 
    f'load_model: "/content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k1/transformer_reverse_enrw2_comet/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/k1/transformer_reverse_enrw2_comet"', f'model_dir: "models/k1/transformer_reverse_enrw2_comet_reload"').replace(
        f'epochs: 60', f'epochs: 3')
        
with open("joeynmt/configs/k1/transformer_reverse_enrw2_comet_reload.yaml",'w') as f:
    f.write(reload_config)

In [ ]:
# Train continued
!cd joeynmt; python3 -m joeynmt train configs/k1/transformer_reverse_enrw2_comet_reload.yaml

2022-09-04 02:27:49,371 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-09-04 02:27:49,415 - INFO - joeynmt.data - Loading training data...
2022-09-04 02:27:50,371 - INFO - joeynmt.data - Building vocabulary...
2022-09-04 02:27:50,899 - INFO - joeynmt.data - Loading dev data...
2022-09-04 02:27:50,922 - INFO - joeynmt.data - Loading test data...
2022-09-04 02:27:51,495 - INFO - joeynmt.data - Data loaded.
2022-09-04 02:27:51,495 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-09-04 02:27:51,848 - INFO - joeynmt.model - Enc-dec model built.
2022-09-04 02:27:54,507 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-09-04 02:27:54,795 - INFO - joeynmt.training - Total params: 12147712
2022-09-04 02:27:58,219 - INFO - joeynmt.training - Loading model from /content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k1/transformer_reverse_enrw2_comet/61200.ckpt
2022-09-04 02:27:59,967 - INFO - joeynmt.helpers -                           cfg.name : t

## Iteration 2

### AL Strategy and preprocessing

In [ ]:
comet_data_load('rwtrain.csv', 'rw', 'comet','k1', '2')

/content/gdrive/Shareddrives/Low_Budget_MT
data/rw
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k1/comet
(30000, 2)
                                         source_sentence  \
59487  Everything about the castle ​ — its multicolor...   

                                         target_sentence    scores  
59487  Ibyo bishushanyo byari iby’ibimasa bifite amab... -0.024238  
(5000, 3) (95000, 3)


In [ ]:
split_srctgt('/data/rw/k1/comet/comet_train.csv','en', 'rw', 'comet','k1', '2')

(35000, 3)


In [ ]:
generating_BPE('en', 'rw', 'comet', 'k1','2')

In [ ]:
# Swahili to English
run_model('rw', 'rw','en','transformer_reverse_enrw2_comet','comet','k1','2')
! cd joeynmt; python3 -m joeynmt train configs/k1/transformer_reverse_enrw2_comet.yaml

2022-10-09 20:10:51,212 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-09 20:10:52,020 - INFO - joeynmt.data - Loading training data...
2022-10-09 20:10:52,504 - INFO - joeynmt.data - Building vocabulary...
2022-10-09 20:10:52,738 - INFO - joeynmt.data - Loading dev data...
2022-10-09 20:10:52,753 - INFO - joeynmt.data - Loading test data...
2022-10-09 20:10:53,063 - INFO - joeynmt.data - Data loaded.
2022-10-09 20:10:53,063 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-09 20:10:53,277 - INFO - joeynmt.model - Enc-dec model built.
2022-10-09 20:10:56,241 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-09 20:10:56,747 - INFO - joeynmt.training - Total params: 12144896
2022-10-09 20:11:00,454 - INFO - joeynmt.helpers -                           cfg.name : transformer_reverse_enrw2_comet
2022-10-09 20:11:00,455 - INFO - joeynmt.helpers -                       cfg.data.src : rw
2022-10-09 20:11:00,455 - INFO - joeynmt.helpers -   

###yhat

In [ ]:
yhat('/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k1/comet/comet_rem.csv', 'en', 'rw','k1','2','transformer_reverse_enrw2_comet')

(95000, 3)
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k1/comet/2
/content/gdrive/Shareddrives/Low_Budget_MT


# Kinyarwanda COMET K2


## Iteration 5

### AL Strategy and preprocessing

In [ ]:
comet_data_load('rwtrain.csv', 'rw', 'comet','k2', '2')

/content/gdrive/Shareddrives/Low_Budget_MT
data/rw
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k2/comet
(30000, 2)
                                         source_sentence  \
93794  Everything about the castle ​ — its multicolor...   

                                         target_sentence    scores  
93794  Ibyo bishushanyo byari iby’ibimasa bifite amab... -0.024791  
(20000, 3) (80000, 3)


In [ ]:
split_srctgt('/data/rw/k2/comet/comet_train.csv','en', 'rw', 'comet','k2', '2')

(50000, 3)


In [ ]:
generating_BPE('en', 'rw', 'comet', 'k2','2')

In [ ]:
# Swahili to English
run_model('rw', 'rw','en','transformer_reverse_enrw2_comet','comet','k2','2')
! cd joeynmt; python3 -m joeynmt train configs/k2/transformer_reverse_enrw2_comet.yaml

2022-10-15 01:12:01,150 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-15 01:12:01,944 - INFO - joeynmt.data - Loading training data...
2022-10-15 01:12:02,776 - INFO - joeynmt.data - Building vocabulary...
2022-10-15 01:12:03,006 - INFO - joeynmt.data - Loading dev data...
2022-10-15 01:12:03,021 - INFO - joeynmt.data - Loading test data...
2022-10-15 01:12:03,411 - INFO - joeynmt.data - Data loaded.
2022-10-15 01:12:03,411 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-15 01:12:03,621 - INFO - joeynmt.model - Enc-dec model built.
2022-10-15 01:12:07,155 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-15 01:12:07,457 - INFO - joeynmt.training - Total params: 12148480
2022-10-15 01:12:10,987 - INFO - joeynmt.helpers -                           cfg.name : transformer_reverse_enrw2_comet
2022-10-15 01:12:10,987 - INFO - joeynmt.helpers -                       cfg.data.src : rw
2022-10-15 01:12:10,987 - INFO - joeynmt.helpers -   

# Kinyarwanda COMET K3


## Iteration 5

### AL Strategy and preprocessing

In [ ]:
comet_data_load('rwtrain.csv', 'rw', 'comet','k3', '2')

/content/gdrive/Shareddrives/Low_Budget_MT
data/rw
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k3/comet
(30000, 2)
                                         source_sentence  \
59517  Everything about the castle ​ — its multicolor...   

                                         target_sentence    scores  
59517  Ibyo bishushanyo byari iby’ibimasa bifite amab... -0.024227  
(20000, 3) (80000, 3)


In [ ]:
split_srctgt('/data/rw/k3/comet/comet_train.csv','en', 'rw', 'comet','k3', '2')

(50000, 3)


In [ ]:
generating_BPE('en', 'rw', 'comet', 'k3','2')

In [ ]:
# Swahili to English
run_model('rw', 'rw','en','transformer_reverse_enrw2_comet','comet','k3','2')
! cd joeynmt; python3 -m joeynmt train configs/k3/transformer_reverse_enrw2_comet.yaml

2022-10-15 10:10:14,242 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-15 10:10:14,784 - INFO - joeynmt.data - Loading training data...
2022-10-15 10:10:15,648 - INFO - joeynmt.data - Building vocabulary...
2022-10-15 10:10:15,884 - INFO - joeynmt.data - Loading dev data...
2022-10-15 10:10:15,900 - INFO - joeynmt.data - Loading test data...
2022-10-15 10:10:16,312 - INFO - joeynmt.data - Data loaded.
2022-10-15 10:10:16,313 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-15 10:10:16,531 - INFO - joeynmt.model - Enc-dec model built.
2022-10-15 10:10:19,780 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-15 10:10:20,045 - INFO - joeynmt.training - Total params: 12149504
2022-10-15 10:10:23,681 - INFO - joeynmt.helpers -                           cfg.name : transformer_reverse_enrw2_comet
2022-10-15 10:10:23,681 - INFO - joeynmt.helpers -                       cfg.data.src : rw
2022-10-15 10:10:23,681 - INFO - joeynmt.helpers -   

# Kinyarwanda COMET K4


## Iteration 5

### AL Strategy and preprocessing

In [ ]:
comet_data_load('rwtrain.csv', 'rw', 'comet','k4', '2')

/content/gdrive/Shareddrives/Low_Budget_MT
data/rw
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k4/comet
(30000, 2)
                                         source_sentence  \
59386  Everything about the castle ​ — its multicolor...   

                                         target_sentence   scores  
59386  Ibyo bishushanyo byari iby’ibimasa bifite amab... -0.02494  
(20000, 3) (80000, 3)


In [ ]:
split_srctgt('/data/rw/k4/comet/comet_train.csv','en', 'rw', 'comet','k4', '2')

(50000, 3)


In [ ]:
generating_BPE('en', 'rw', 'comet', 'k4','2')

In [ ]:
# Swahili to English
run_model('rw', 'rw','en','transformer_reverse_enrw2_comet','comet','k4','2')
! cd joeynmt; python3 -m joeynmt train configs/k4/transformer_reverse_enrw2_comet.yaml

2022-10-15 10:11:36,008 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-15 10:11:36,997 - INFO - joeynmt.data - Loading training data...
2022-10-15 10:11:37,901 - INFO - joeynmt.data - Building vocabulary...
2022-10-15 10:11:38,129 - INFO - joeynmt.data - Loading dev data...
2022-10-15 10:11:38,146 - INFO - joeynmt.data - Loading test data...
2022-10-15 10:11:38,537 - INFO - joeynmt.data - Data loaded.
2022-10-15 10:11:38,537 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-15 10:11:38,753 - INFO - joeynmt.model - Enc-dec model built.
2022-10-15 10:11:42,238 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-15 10:11:42,490 - INFO - joeynmt.training - Total params: 12147968
2022-10-15 10:11:45,966 - INFO - joeynmt.helpers -                           cfg.name : transformer_reverse_enrw2_comet
2022-10-15 10:11:45,966 - INFO - joeynmt.helpers -                       cfg.data.src : rw
2022-10-15 10:11:45,966 - INFO - joeynmt.helpers -   

In [ ]:
# Output our validation accuracy
! cat "joeynmt/models/k4/transformer_reverse_enrw2_comet/validations.txt"

Steps: 600	Loss: 123075.10156	PPL: 95.16739	bleu: 0.06540	LR: 0.00100000	*
Steps: 1200	Loss: 110747.23438	PPL: 60.29930	bleu: 0.66971	LR: 0.00100000	*
Steps: 1800	Loss: 100348.67188	PPL: 41.03464	bleu: 1.55332	LR: 0.00100000	*
Steps: 2400	Loss: 92952.75000	PPL: 31.20741	bleu: 2.73622	LR: 0.00100000	*
Steps: 3000	Loss: 88187.25781	PPL: 26.16076	bleu: 3.58090	LR: 0.00100000	*
Steps: 3600	Loss: 84989.57812	PPL: 23.24053	bleu: 4.10767	LR: 0.00100000	*
Steps: 4200	Loss: 82307.57812	PPL: 21.04417	bleu: 4.79675	LR: 0.00100000	*
Steps: 4800	Loss: 80152.29688	PPL: 19.43053	bleu: 5.35289	LR: 0.00100000	*
Steps: 5400	Loss: 78218.08594	PPL: 18.08803	bleu: 6.13653	LR: 0.00100000	*
Steps: 6000	Loss: 76539.64062	PPL: 16.99846	bleu: 6.61622	LR: 0.00100000	*
Steps: 6600	Loss: 75379.23438	PPL: 16.28379	bleu: 6.80432	LR: 0.00100000	*
Steps: 7200	Loss: 73775.14062	PPL: 15.34507	bleu: 7.35212	LR: 0.00100000	*
Steps: 7800	Loss: 72953.33594	PPL: 14.88532	bleu: 7.63259	LR: 0.00100000	*
Steps: 8400	Loss: 71884

In [ ]:
# Reloading configuration file
ckpt_number = 61200
config = run_model('rw', 'rw','en','transformer_reverse_enrw2_comet','comet','k4','2')
reload_config = config.replace(
    f'#load_model: path ', 
    f'load_model: "/content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k4/transformer_reverse_enrw2_comet/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/k4/transformer_reverse_enrw2_comet"', f'model_dir: "models/k4/transformer_reverse_enrw2_comet_reload"').replace(
        f'epochs: 60', f'epochs: 3')
        
with open("joeynmt/configs/k4/transformer_reverse_enrw2_comet_reload.yaml",'w') as f:
    f.write(reload_config)

In [ ]:
# Train continued
!cd joeynmt; python3 -m joeynmt train configs/k4/transformer_reverse_enrw2_comet_reload.yaml

2022-10-16 19:35:19,867 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-16 19:35:20,266 - INFO - joeynmt.data - Loading training data...
2022-10-16 19:35:25,894 - INFO - joeynmt.data - Building vocabulary...
2022-10-16 19:35:27,284 - INFO - joeynmt.data - Loading dev data...
2022-10-16 19:35:28,323 - INFO - joeynmt.data - Loading test data...
2022-10-16 19:35:30,033 - INFO - joeynmt.data - Data loaded.
2022-10-16 19:35:30,034 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-16 19:35:30,238 - INFO - joeynmt.model - Enc-dec model built.
2022-10-16 19:35:33,576 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-16 19:35:33,835 - INFO - joeynmt.training - Total params: 12147968
2022-10-16 19:35:37,203 - INFO - joeynmt.training - Loading model from /content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k4/transformer_reverse_enrw2_comet/61200.ckpt
2022-10-16 19:35:41,323 - INFO - joeynmt.helpers -                           cfg.name : t

# Kinyarwanda COMET K5


## Iteration 5

### AL Strategy and preprocessing

In [ ]:
comet_data_load('rwtrain.csv', 'rw', 'comet','k5', '2')

/content/gdrive/Shareddrives/Low_Budget_MT
data/rw
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k5/comet
(30000, 2)
                                         source_sentence  \
59398  Everything about the castle ​ — its multicolor...   

                                         target_sentence    scores  
59398  Ibyo bishushanyo byari iby’ibimasa bifite amab... -0.024339  
(20000, 3) (80000, 3)


In [ ]:
split_srctgt('/data/rw/k5/comet/comet_train.csv','en', 'rw', 'comet','k5', '2')

(50000, 3)


In [ ]:
generating_BPE('en', 'rw', 'comet', 'k5','2')

In [ ]:
# Swahili to English
run_model('rw', 'rw','en','transformer_reverse_enrw2_comet','comet','k5','2')
! cd joeynmt; python3 -m joeynmt train configs/k5/transformer_reverse_enrw2_comet.yaml

2022-10-15 10:18:26,895 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-15 10:18:27,359 - INFO - joeynmt.data - Loading training data...
2022-10-15 10:18:28,224 - INFO - joeynmt.data - Building vocabulary...
2022-10-15 10:18:28,475 - INFO - joeynmt.data - Loading dev data...
2022-10-15 10:18:28,491 - INFO - joeynmt.data - Loading test data...
2022-10-15 10:18:28,888 - INFO - joeynmt.data - Data loaded.
2022-10-15 10:18:28,888 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-15 10:18:29,096 - INFO - joeynmt.model - Enc-dec model built.
2022-10-15 10:18:32,635 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-15 10:18:32,901 - INFO - joeynmt.training - Total params: 12147968
2022-10-15 10:18:36,537 - INFO - joeynmt.helpers -                           cfg.name : transformer_reverse_enrw2_comet
2022-10-15 10:18:36,538 - INFO - joeynmt.helpers -                       cfg.data.src : rw
2022-10-15 10:18:36,538 - INFO - joeynmt.helpers -   

In [ ]:
!cd joeynmt; python -m joeynmt test 'models/k5/transformer_reverse_enrw2_comet/config.yaml'

2022-10-16 19:57:57,312 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-16 19:57:57,318 - INFO - joeynmt.data - Building vocabulary...
2022-10-16 19:57:59,667 - INFO - joeynmt.data - Loading dev data...
2022-10-16 19:58:00,286 - INFO - joeynmt.data - Loading test data...
2022-10-16 19:58:01,653 - INFO - joeynmt.data - Data loaded.
2022-10-16 19:58:01,718 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 3600
2022-10-16 19:58:01,718 - INFO - joeynmt.prediction - Loading model from models/k5/transformer_reverse_enrw2_comet/63000.ckpt
2022-10-16 19:58:08,925 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-16 19:58:09,139 - INFO - joeynmt.model - Enc-dec model built.
2022-10-16 19:58:09,210 - INFO - joeynmt.prediction - Decoding on dev set (/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k5/comet/2/dev.bpe.en)...
2022-10-16 19:58:47,027 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period ('.')
